In [1]:
#установка библиотек
!pip install -U psycopg2
!pip install -U plotly

In [2]:
import psycopg2
psycopg2.__version__

'2.9.3 (dt dec pq3 ext lo64)'

# проверка подключения

In [3]:
# student11
# student11_password

In [4]:
#проверка подключения PostgreSQL 
import psycopg2
import pandas as pd
#Библиотека ждя визуализации
from IPython.display import HTML
import plotly.express as px


#!введите свои реквизиты!
DB_HOST = '52.157.159.24'
DB_USER = 'student11'
DB_USER_PASSWORD = 'student11_password'
DB_NAME = 'sql_ex_third_stream'

conn = psycopg2.connect(host=DB_HOST, user=DB_USER, password=DB_USER_PASSWORD, dbname=DB_NAME)

In [5]:
conn

<connection object at 0x7f2dd4763c20; dsn: 'user=student11 password=xxx dbname=sql_ex_third_stream host=52.157.159.24', closed: 0>

In [6]:
conn.isexecuting()

False

# задания












# task1 (lesson5)
* Компьютерная фирма: Сделать view (pages_all_products), в которой будет постраничная разбивка всех продуктов (не более двух продуктов на одной странице)
* Вывод: все данные из laptop, номер страницы, список всех страниц

sample:
* 1 1
* 2 1
* 1 2
* 2 2
* 1 3
* 2 3










In [7]:
SQL1 =  """
            DROP VIEW IF EXISTS pages_all_products
            ;

            CREATE VIEW pages_all_products AS
            (
                WITH row_number_subquery AS
                (
                    SELECT *, ROW_NUMBER() OVER (ORDER BY price) AS rn
                    FROM laptop
                ),
                page_number_subquery AS
                (
                    SELECT *, (rn+1) / 2 AS page_number
                    FROM row_number_subquery
                )
                SELECT *, ROW_NUMBER(*) OVER (PARTITION BY page_number) AS number_in_page
                FROM page_number_subquery
            )
            ;

            SELECT * 
            FROM pages_all_products
            ;
        """

df = pd.read_sql_query(SQL1, conn)
df


,code,model,speed,ram,hd,price,screen,rn,page_number,number_in_page
0,1,1298,350,32,4.0,700.0,11,1,1,1
1,6,1298,450,64,10.0,950.0,12,2,1,2
2,2,1321,500,64,8.0,970.0,12,3,2,1
3,4,1298,600,64,10.0,1050.0,15,4,2,2
4,5,1752,750,128,10.0,1150.0,14,5,3,1
5,3,1750,750,128,12.0,1200.0,14,6,3,2


# task2 (lesson5)
* Компьютерная фирма: Сделать view (distribution_by_type), в рамках которого будет процентное соотношение всех товаров по типу устройства. Вывод: производитель, тип, процент (%)

In [8]:
SQL1 =  """
            DROP VIEW IF EXISTS distribution_by_type
            ;

            CREATE VIEW distribution_by_type AS
            (
                SELECT DISTINCT maker, type, 
                    CAST (100 * COUNT(1) OVER (PARTITION BY type) / (SELECT COUNT(1) FROM product) AS FLOAT) AS percent
                FROM product
                ORDER BY type, maker
            )
            ;

            SELECT * 
            FROM distribution_by_type
            ;
        """

df = pd.read_sql_query(SQL1, conn)
df


,maker,type,percent
0,A,Laptop,21.0
1,B,Laptop,21.0
2,C,Laptop,21.0
3,A,PC,47.0
4,B,PC,47.0
5,E,PC,47.0
6,A,Printer,31.0
7,D,Printer,31.0
8,E,Printer,31.0


# task3 (lesson5)
* Компьютерная фирма: Сделать на базе предыдущенр view график - круговую диаграмму. Пример https://plotly.com/python/histograms/

In [9]:
# fig = px.pie(df, values='percent', names='type', title='distribution_by_type')
# fig.show()

# task4 (lesson5)
* Корабли: Сделать копию таблицы ships (ships_two_words), но название корабля должно состоять из двух слов

In [10]:
SQL1 =  """
            DROP TABLE IF EXISTS ships_two_words_11;

            CREATE TABLE ships_two_words_11 AS
                SELECT * 
                FROM ships
                WHERE (name LIKE '% %')
            ;

            SELECT * FROM ships_two_words_11;
        """

df = pd.read_sql_query(SQL1, conn)
df


,name,class,launched
0,New Jersey,Iowa,1943
1,North Carolina,North Carolina,1941
2,Royal Oak,Revenge,1916
3,Royal Sovereign,Revenge,1916
4,South Dakota,North Carolina,1941


# task5 (lesson5)
* Корабли: Вывести список кораблей, у которых class отсутствует (IS NULL) и название начинается с буквы "S"

In [11]:
SQL1 =  """
            SELECT * 
            FROM ships
            WHERE (name LIKE 'S%') AND class IS NULL
            ;
        """

df = pd.read_sql_query(SQL1, conn)
df


,name,class,launched


# task6 (lesson5)
* Компьютерная фирма: Вывести все принтеры производителя = 'A' со стоимостью выше средней по принтерам производителя = 'C' и три самых дорогих (через оконные функции). Вывести model

In [12]:
SQL1 =  """
            WITH maker_price AS
            (
                SELECT product.maker, product.model, price
                FROM product
                INNER JOIN printer
                ON product.model = printer.model
            ),
            maker_C_avg_price AS
            (
                SELECT AVG(price) OVER (PARTITION BY maker) 
                AS avg_price
                FROM maker_price 
                WHERE maker = 'C'
            ),
            -- у производителя C отсутствуют принтеры
            maker_C_avg_price_single_value AS
            (
                SELECT (SELECT avg_price FROM maker_C_avg_price) AS avg_price_single_value
            ),
            maker_C_avg_price_corrected AS
            (
                SELECT CASE
                WHEN avg_price_single_value IS NULL
                    THEN 0
                    ELSE avg_price_single_value
                END avg_price_corrected
                FROM maker_C_avg_price_single_value 
            ),
            top_price AS
            (
                SELECT model, price, ROW_NUMBER() OVER (ORDER BY price DESC) AS rn
                FROM maker_price
            )

            SELECT model
            FROM maker_price
            WHERE maker = 'A' AND price > (SELECT avg_price_corrected FROM maker_C_avg_price_corrected)

            UNION 
            
            SELECT model
            FROM top_price
            WHERE rn <= 3
        """

df = pd.read_sql_query(SQL1, conn)
df



,model
0,1401
1,1288
2,1434
3,1408
4,1276
